In [ ]:
pip install -qU langchain_openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "03-01"

# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessageChunk
from pydantic import BaseModel, Field

# LLM 설정
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# 텍스트 파일에서 이메일 내용 읽기
file_path = "/content/data/email.txt"
with open(file_path, "r", encoding="utf-8") as file:
    email_conversation = file.read()

# 실시간 출력 함수 정의
def stream_response(response, return_output=False):
    answer = ""
    for token in response:
        if isinstance(token, AIMessageChunk):
            answer += token.content
            print(token.content, end="", flush=True)
        elif isinstance(token, str):
            answer += token
            print(token, end="", flush=True)
    if return_output:
        return answer

# Pydantic 모델 정의
class EmailSummary(BaseModel):
    person: str = Field(description="The sender of the email")
    email: str = Field(description="The email address of the sender")
    subject: str = Field(description="The subject of the email")
    summary: str = Field(description="A summary of the email content")
    is_spam: bool = Field(description="Whether the email is spam or not, 만약 '스팸'이 아니면 'not spam'이라고 입력해")
    date: str = Field(
        description="The meeting date and time mentioned in the email content"
    )

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# 프롬프트 정의
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# PydanticOutputParser를 추가해 프롬프트 포맷 설정
prompt = prompt.partial(format=parser.get_format_instructions())

# 체인 생성
chain = prompt | llm | parser

# 체인 실행 및 결과 출력
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "Extract the main content of the email.",
    }
)

# 결과 출력
print(response)
